In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=0, load_data=False)
context = Context('amended-2-14', debug=0, dep_var=dep_var2, mulr=14, load_data=False)

# data

In [ ]:
# list existing data files
print('Existing data files:', [o.stem for o in Path(context.fn_feather).parent.glob('*.feather')])

fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation
# fmtr.drop_redundent_columns() # already done for cycles
idx = fmtr.beginning_index()

target data format: `[flattened matrix: 30x150, extra cont: 1x2, extra cat: n]`

parameters: cat_names

Use no cat for now

In [ ]:
df_conts, stat_x = fmtr.get_x(normalize=True)
extra_x = fmtr.get_extra_x_task2()
deps = fmtr.get_y()
stat_y = [o.loc[context.dep_var] for o in stat_x]
stat_extra_x = [o.loc[dep_var1] for o in stat_x]
train_cont = flatten_and_cat(df_conts, [extra_x, deps], sl=context.sl)

In [ ]:
train_cont.to_feather(context.fn_feather)
np.savez_compressed(context.fn_np, idx=idx, stat_x=stat_x, stat_y=stat_y, stat_extra_x=stat_extra_x)
context.fn_feather

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1)
context = Context('amended-2-14', debug=0, dep_var=dep_var2, mulr=14)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
x, y = ni(databunch.train_dl)
[o.shape for o in x], y.shape

# model

In [ ]:
use_extra_x = False
hyper_params = {
    'n_hidden': 400,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.4,
    'weight_p': 0.5,
    'layers': [3*(context.n_cont + int(use_extra_x) * 2), 5*context.n_cont, 5, len(context.dep_var)],
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}

In [ ]:
class PoolingLinearClassifier(nn.Module):                                                                                                                                                    
    "Create a linear classifier with pooling."                                                                                                                                               
                                                                                                                                                                                             
    def __init__(self, layers:Collection[int], drops:Collection[float]):                                                                                                                     
        super().__init__()                                                                                                                                                                   
        mod_layers = []                                                                                                                                                                      
        activs = [nn.ReLU(inplace=True)] * (len(layers) - 2) + [None]                                                                                                                        
        for n_in,n_out,p,actn in zip(layers[:-1],layers[1:], drops, activs):                                                                                                                 
            mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)                                                                                                                           
        self.layers = nn.Sequential(*mod_layers)                                                                                                                                             
                                                                                                                                                                                             
    def pool(self, x:Tensor, bs:int, is_max:bool):                                                                                                                                           
        "Pool the tensor along the seq_len dimension."                                                                                                                                       
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d                                                                                                                       
        return f(x.transpose(1,2), (1,)).view(bs,-1)                                                                                                                                         
                                                                                                                                                                                             
    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:                                                                                                              
        raw_outputs, outputs = input                                                                                                                                                         
        output = outputs[-1]                                                                                                                                                                 
        bs,sl,_ = output.size()                                                                                                                                                              
        avgpool = self.pool(output, bs, False)                                                                                                                                               
        mxpool = self.pool(output, bs, True)                                                                                                                                                 
        x = torch.cat([output[:,-1], mxpool, avgpool], 1)                                                                                                                                    
        x = self.layers(x)                                                                                                                                                                   
        return x, raw_outputs, outputs

In [ ]:
class Task2Model(AWD_LSTM):
    def __init__(self, n_cont, emb_sz:int, n_hid:int, n_layers:int, layers:Collection[int], drops:Collection[float], hidden_p:float=0.2,
                 input_p:float=0.6, embed_p:float=0.1, weight_p:float=0.5, bidir:bool=False, 
                 sl=30):
        # note: check pad_token when generating cat variables
        qrnn = False # continuous variables only for this model
        vocab_sz,pad_token=1,0 # temp
        super().__init__(vocab_sz, n_cont, n_hid, n_layers, pad_token, hidden_p, input_p, embed_p, weight_p, qrnn, bidir)
        self.sl,self.n_cont,self.emb_sz = sl,n_cont,emb_sz
        self.pool_linear_classifier = PoolingLinearClassifier(layers, drops)
        
        # re-initialize embedding layer
        vocab_sz,pad_token=1,0
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        
    def forward(self, x_cat:Tensor, x_cont:Tensor):
        bs,*_ = x_cat.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        x_cont, x_extra = x_cont[:,:self.n_cont*self.sl], x_cont[:,self.n_cont*self.sl:]
        x_cont = x_cont.view(bs, self.sl, self.n_cont)
        x_cat = self.encoder_dp(x_cat)

        raw_output = self.input_dp(x_cont)
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
        self.hidden = to_detach(new_hidden, cpu=False)
        
        # fc
        x, raw_outputs, outputs = self.pool_linear_classifier((raw_outputs, outputs))
        return x, raw_outputs, outputs
    
    def _one_hidden(self, l:int)->Tensor:
        "Return one hidden state."
        nh = (self.n_hid if l != self.n_layers - 1 else self.n_cont) // self.n_dir
        return one_param(self).new(1, self.bs, nh).zero_()
    
emb_sz = 3 # use rule of thumb later
model = Task2Model(context.n_cont,emb_sz, hyper_params['n_hidden'], hyper_params['n_layers'], hyper_params['layers'], hyper_params['drops'], 
                  hidden_p=hyper_params['hidden_p'], input_p=hyper_params['input_p'], weight_p=hyper_params['weight_p'])
model = model.cuda()

In [ ]:
p, raw_outputs, outputs = model(*x)

In [ ]:
RNNTrainer?

In [ ]:
learner = get_new_model(context, databunch, hyper_params)

In [ ]:
x, y = ni(learner.data.train_dl)

In [ ]:
learner.lr_find()

In [ ]:
learner.metrics[0].stats

In [ ]:
learner.recorder.plot()

In [ ]:
learner.fit(10, lr=1e-1)

In [ ]:
learner.fit(20, lr=1e-1)

In [ ]:
learner.fit(20, lr=1e-2)

In [ ]:
learner.fit(20, lr=1e-3)

In [ ]:
learner.fit(20, lr=1e-6)

In [ ]:
learner.fit(40, lr=1e-7)

In [ ]:
context.cyc_cont.iloc[:,-4:]

In [ ]:
learner.save('new_model_test')

In [ ]:
npz = np.load(context.fn_np)

In [ ]:
npz['stat_y']

In [ ]:
extra = denormalize(context.cyc_cont.iloc[:,-4:-2], *npz['stat_extra_x'])

In [ ]:
extra.mean(), extra.std()